<a href="https://colab.research.google.com/github/natdebandi/text_mining_unrn/blob/main/topic_modelling_ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TEXT MINING

UNRN

Especialización en ciencias de datos aplicada

Natalia Debandi

ndebandi@unrn.edu.ar

**CLASE 4**

Aprendizaje no supervisado - Topic modelling

Ejercicio

In [1]:
!pip install gensim
#Si les llega a falla la siguiente linea ejecuten esto y reinicien la sesion. A veces falla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 884.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [2]:
!pip install feedparser beautifulsoup4 requests nltk gensim matplotlib wordcloud fake-useragent pyldavis

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=e3d1ffae3f43ae54103dd2031c710a088621e32d66629c8130bd3553a576dca7
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import feedparser
from bs4 import BeautifulSoup
import requests
import re
from fake_useragent import UserAgent
import time
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Configuración inicial
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Voy a armar un dataset para hacer LDA a partir de google news

In [6]:
def fetch_article_content(url, timeout=10):
    """Improved content extraction with better error handling"""
    try:
        headers = {'User-Agent': ua.random}
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises exception for 4XX/5XX errors
        except requests.exceptions.RequestException as e:
            print(f"Request failed for {url}: {e}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Try different strategies to get article content
        strategies = [
            lambda: soup.find('article'),
            lambda: soup.find(class_=re.compile('article|content|body|post', re.I)),
            lambda: soup.find(itemprop='articleBody'),
            lambda: soup.find('main'),
            lambda: soup.find('body')
        ]

        article_content = None
        for strategy in strategies:
            article_content = strategy()
            if article_content:
                break

        if not article_content:
            return None

        # Clean up the content
        for element in article_content(['script', 'style', 'iframe', 'nav', 'footer', 'aside', 'header']):
            element.decompose()

        text = ' '.join([p.get_text() for p in article_content.find_all(['p', 'h1', 'h2', 'h3'])])
        text = re.sub(r'\s+', ' ', text).strip()
        return text[:10000] if text else None

    except Exception as e:
       # print(f"Error processing {url}: {e}")
        return None


In [24]:
def extraer_noticias_completas(query="Argentine", num_noticias=15):
    """More reliable news extraction function"""
    base_url = f"https://news.google.com/rss/search?q={query.replace(' ', '+')}&hl=en-419&gl=US&ceid=US:es-419"
    #base_url = f"https://news.google.com/rss/search?q={query.replace(' ', '+')}&hl=es-419&gl=AR&ceid=AR:es-419"

    try:
        feed = feedparser.parse(base_url)
        if not feed.entries:
            print("No news entries found in the RSS feed")
            return pd.DataFrame()

        noticias = []
        for i, entry in enumerate(feed.entries[:num_noticias]):
            #print(f"Processing {i+1}/{min(len(feed.entries), num_noticias)}: {entry.title[:50]}...")

            content = None
            if hasattr(entry, 'link'):
                content = fetch_article_content(entry.link)
                time.sleep(1)  # Be polite with requests

            if content:
                noticias.append({
                    'titulo': entry.title,
                    'fuente': entry.source.title if hasattr(entry, 'source') else 'Desconocido',
                    'fecha': entry.published if hasattr(entry, 'published') else None,
                    'enlace': entry.link,
                    'contenido': content
                })
            else:
                noticias.append({
                    'titulo': entry.title,
                    'fuente': entry.source.title if hasattr(entry, 'source') else 'Desconocido',
                   'fecha': entry.published if hasattr(entry, 'published') else None,
                    'enlace': entry.link,
                    'contenido': entry.title
                })

        return pd.DataFrame(noticias)

    except Exception as e:
        print(f"Error in news extraction: {e}")
        return pd.DataFrame()



In [25]:
# Extraer noticias (reducido a 30 para demo)
print("Extrayendo noticias con contenido completo...")
#df_noticias = extraer_noticias_completas()
df_noticias = extraer_noticias_completas("Argentine", 30)
print(f"\nNoticias extraídas correctamente: {len(df_noticias)}")
#print(f"Ejemplo de contenido:\n{df_noticias['contenido'].iloc[0][:300]}...")

Extrayendo noticias con contenido completo...

Noticias extraídas correctamente: 30


In [12]:
def preprocesar_texto(texto):
    # Tokenización y limpieza
    stop_words = set(stopwords.words('spanish') + list(stopwords.words('english')))
    lemmatizer = WordNetLemmatizer()

    # Eliminar URLs y caracteres especiales
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@\w+|\#|\d+', '', texto)

    tokens = word_tokenize(texto.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens
              if token.isalpha() and token not in stop_words and len(token) > 3]

    return tokens

In [15]:
# Aplicar preprocesamiento
print("\nPreprocesando texto...")
df_noticias['tokens'] = df_noticias['contenido'].apply(preprocesar_texto)
df_noticias['texto_limpio'] = df_noticias['tokens'].apply(lambda x: ' '.join(x))


Preprocesando texto...


In [16]:
df_noticias

,titulo,fuente,fecha,enlace,contenido,tokens,texto_limpio
0,El directorio del FMI no logró llegar a un acu...,Ámbito Financiero,"Fri, 04 Apr 2025 21:47:00 GMT",https://news.google.com/rss/articles/CBMi0gFBV...,El directorio del FMI no logró llegar a un acu...,"[directorio, logró, llegar, acuerdo, primer, d...",directorio logró llegar acuerdo primer desembo...
1,La Argentina y el FMI discuten un desembolso i...,Infobae,"Fri, 04 Apr 2025 23:03:00 GMT",https://news.google.com/rss/articles/CBMi8wFBV...,La Argentina y el FMI discuten un desembolso i...,"[argentina, discuten, desembolso, inicial, may...",argentina discuten desembolso inicial mayor es...
2,Claver Carone explosivo contra Milei: exigió q...,La Política Online,"Thu, 03 Apr 2025 22:51:06 GMT",https://news.google.com/rss/articles/CBMi0gFBV...,Claver Carone explosivo contra Milei: exigió q...,"[claver, carone, explosivo, milei, exigió, arg...",claver carone explosivo milei exigió argentina...
3,Argentina buscará ante el local Colombia asegu...,Infobae,"Sat, 05 Apr 2025 09:55:38 GMT",https://news.google.com/rss/articles/CBMizAFBV...,Argentina buscará ante el local Colombia asegu...,"[argentina, buscará, local, colombia, asegurar...",argentina buscará local colombia asegurar pasa...
4,"Nos volvimos a ilusionar: Argentina, a un paso...",Olé,"Sat, 05 Apr 2025 00:38:05 GMT",https://news.google.com/rss/articles/CBMioAFBV...,"Nos volvimos a ilusionar: Argentina, a un paso...","[volvimos, ilusionar, argentina, paso, mundial...",volvimos ilusionar argentina paso mundial qatar
...,...,...,...,...,...,...,...
95,El impacto Trump en la Argentina: los 10 alime...,Clarin.com,"Thu, 03 Apr 2025 23:12:00 GMT",https://news.google.com/rss/articles/CBMixwFBV...,El impacto Trump en la Argentina: los 10 alime...,"[impacto, trump, argentina, alimentos, país, v...",impacto trump argentina alimentos país vende u...
96,Suba de aranceles de EEUU: ¿puede afectar el r...,Ámbito Financiero,"Thu, 03 Apr 2025 17:57:00 GMT",https://news.google.com/rss/articles/CBMirAFBV...,Suba de aranceles de EEUU: ¿puede afectar el r...,"[suba, aranceles, eeuu, afectar, regreso, arge...",suba aranceles eeuu afectar regreso argentina ...
97,Hackearon a un proveedor de software médico en...,La Voz del Interior,"Sat, 05 Apr 2025 10:39:27 GMT",https://news.google.com/rss/articles/CBMi0wFBV...,Hackearon a un proveedor de software médico en...,"[hackearon, proveedor, software, médico, argen...",hackearon proveedor software médico argentina ...
98,"Exportadores, industriales y las economías reg...",elDiarioAR.com,"Fri, 04 Apr 2025 12:40:58 GMT",https://news.google.com/rss/articles/CBMi2wFBV...,"Exportadores, industriales y las economías reg...","[exportadores, industriales, economías, region...",exportadores industriales economías regionales...


In [26]:
df_noticias.head()

,titulo,fuente,fecha,enlace,contenido
0,Instituto y Newell’s avanzaron a la próxima fa...,Infobae,"Fri, 04 Apr 2025 00:49:32 GMT",https://news.google.com/rss/articles/CBMi4AFBV...,Instituto y Newell’s avanzaron a la próxima fa...
1,Instituto goleó a Deportivo Madryn en la Copa ...,Olé,"Thu, 03 Apr 2025 21:25:19 GMT",https://news.google.com/rss/articles/CBMingFBV...,Instituto goleó a Deportivo Madryn en la Copa ...
2,Newell’s venció por penales a Kimberley y avan...,TV Pública,"Fri, 04 Apr 2025 14:16:49 GMT",https://news.google.com/rss/articles/CBMipAFBV...,Newell’s venció por penales a Kimberley y avan...
3,Instituto busca seguir por la buena senda por ...,VAVEL.com,"Fri, 04 Apr 2025 17:41:48 GMT",https://news.google.com/rss/articles/CBMi4AFBV...,Instituto busca seguir por la buena senda por ...
4,"Goleó a Madryn. Con La Agustina de bandera, In...",La Voz del Interior,"Fri, 04 Apr 2025 10:00:00 GMT",https://news.google.com/rss/articles/CBMitwFBV...,"Goleó a Madryn. Con La Agustina de bandera, In..."


In [17]:
def modelado_lda(textos, num_topics=5):
    diccionario = Dictionary(textos)
    corpus = [diccionario.doc2bow(texto) for texto in textos]

    modelo = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=42,
        passes=15,
        alpha='auto'
    )

    return modelo, corpus, diccionario

print("\nEntrenando modelo LDA...")
modelo_lda, corpus, diccionario = modelado_lda(df_noticias['tokens'])


Entrenando modelo LDA...


Vamos a usar una visualizaicon interactiva

In [18]:
print("\nGenerando visualización interactiva...")
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(modelo_lda, corpus, diccionario)
vis


Generando visualización interactiva...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.110521 -0.040510       1        1  33.676284
0      0.061705 -0.064836       2        1  21.089006
3      0.068207 -0.008852       3        1  17.719415
1      0.002172  0.109109       4        1  16.219873
2     -0.021562  0.005088       5        1  11.295423, topic_info=          Term       Freq      Total Category  logprob  loglift
14     infobae  15.000000  15.000000  Default  30.0000  30.0000
1    argentina  65.000000  65.000000  Default  29.0000  29.0000
130  economist   3.000000   3.000000  Default  28.0000  28.0000
222   historia   2.000000   2.000000  Default  27.0000  27.0000
221      deuda   1.000000   1.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
50      javier   0.583856   2.960833   Topic5  -5.1817   0.5572
48        dijo   0.583856   1.668501   Topic5  -5.1817   1.1307
229     sector   0.583856   2.242187   Topic5  -5.1817   0.8352
68     cayeron   0.583855   1.827169   Topic5  -5.1817   1.0399
56     impacto   0.583855   2.464068   Topic5  -5.1817   0.7409

[282 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
303       1  0.748710     abril
303       3  0.374355     abril
532       3  0.832234     abrió
67        1  0.633358  acciones
67        3  0.316679  acciones
...     ...       ...       ...
9         2  0.196274    ámbito
9         3  0.196274    ámbito
9         4  0.196274    ámbito
316       5  0.915450    éxitos
188       3  0.556329   últimas

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 2, 3])